### Saha for N Ionization

The ionization energies are obtained from [NIST](https://physics.nist.gov/cgi-bin/ASD/ie.pl?spectra=Xe&units=1&at_num_out=on&el_name_out=on&seq_out=on&shells_out=on&level_out=on&e_out=0&unc_out=on&biblio=on)

The statistical weights are approximately derived also with NIST, using g_i = 2 J_i + 1 of the ground level of that state


In [1]:
import numpy as np
from hnc.hnc.constants import *

from saha.core.saha import plasma, calculate_ionization_fractions

from saha.core.table_generator import saha_table

import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 14})
plt.rcParams["figure.figsize"] = (6,4)
plt.rcParams["figure.dpi"] = 200
plt.rcParams['xtick.labelsize']=14

color_smooth = lambda N: plt.cm.viridis(np.linspace(0,1,N))
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']


ModuleNotFoundError: No module named 'mendeleev'


In [2]:
# Zach Johnson 6/27/24
# Xenon 5 bar experiment from UCLA

nn_invcc_at_Pbar_TK = lambda Pbar, TK: Pbar*bar_to_AU/(TK*K_to_AU)*AU_to_invcc

Ar_nn_invcc = nn_invcc_at_Pbar_TK(25, 290)
Ar_TK_peak = 17.761029411764707e3 # 0.008097165991902834 ns?

# https://physics.nist.gov/cgi-bin/ASD/ie.pl?spectra=Ar&units=1&at_num_out=on&el_name_out=on&seq_out=on&shells_out=on&level_out=on&e_out=0&unc_out=on&biblio=on
Ar_ionization_energies_AU = np.array([0, 15.7596119, 27.62967, 40.735, 59.58, 74.84, 91.290])*eV_to_AU
Ar_J_ground_level = np.array([0, 3/2, 2, 3/2, 0, 1/2, 0])
Ar_ionization_degeneracies = 2*Ar_J_ground_level + 1
# Ar_ionization_degeneracies = np.array([1, 5.63, 9.03])

# Input: Number of ionizations
Z = 18
Ar25bar_plasma = plasma("Ar25bar_weak-coupling_Δχ", Z, Ar_ionization_energies_AU, Ar_ionization_degeneracies)

# Temperature density
Zbar_guess = 0.1
nn_invcc = 1e20 / Zbar_guess
nn_AU = nn_invcc  * invcc_to_AU

ne_invcc_from_Zbar = lambda Zbar: nn_invcc*Zbar

T_AU  = 2*eV_to_AU
print(Ar25bar_plasma.name)
# Calculate ionization fractions



Ar25bar_weak-coupling_Δχ


In [3]:
Ar_nn_invcc

6.243944291216209e+20

In [94]:
from scipy.optimize import minimize_scalar

def ne_from_nn(nn_invcc):
    nn_AU = nn_invcc*invcc_to_AU
    Zbar, xi_array, χ_array = calculate_ionization_fractions(Ar25bar_plasma, nn_AU, T_AU, IPD=True, N_ions=2)
    return nn_invcc * Zbar

def f_to_min(nn_invcc):
    return np.abs(1e20 - ne_from_nn(nn_invcc))
    
Fontes_sol = minimize_scalar(f_to_min, bounds=(1e19, 1e21))
Fontes_sol

 message: Solution found.
 success: True
  status: 0
     fun: 416484392960.0
       x: 8.470088865469637e+19
     nit: 31
    nfev: 31

In [121]:
Fontes_sol.x, calculate_ionization_fractions(Ar25bar_plasma, 1e20*invcc_to_AU, T_AU, IPD=True, N_ions=5)

(8.470088865469637e+19,
 (1.2600281728658462,
  array([0.33684333, 0.21538581, 0.30092766, 0.14509459, 0.00123976,
         0.00050884]),
  array([0.        , 0.49557073, 0.84820298, 1.24623064, 1.8551873 ,
         2.33239763])))

In [74]:
calculate_ionization_fractions(Ar25bar_plasma, 1e21*invcc_to_AU, T_AU, IPD=False, N_ions=2)

(0.22219115075615573,
 array([0.82165898, 0.13449088, 0.04385013]),
 array([0.        , 0.57915511, 1.01537173]))

In [70]:
np.exp(-Ar_ionization_energies_AU / T_AU)[2]/np.exp(-Ar_ionization_energies_AU / T_AU)[1]

0.002645145911936588